In [1]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [2]:
from utilities import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [5]:
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))
df_feats_icd_transform = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_icd_transform.pkl','rb'))

df_icu_graph_features = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_icu_graph_features.pkl','rb'))

df_feats_ppg_train_all = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_train_separate.pkl','rb'))
df_feats_ppg_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_val_separate.pkl','rb'))
df_feats_ppg_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_test.pkl','rb'))


feats_n2v_all_train = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_train.pkl','rb'))
feats_n2v_all_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_val.pkl','rb'))
feats_n2v_all_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_test.pkl','rb'))



Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels.pkl','rb'))

In [6]:
df_feats_clinical_icd = pd.merge(df_feats_clinical,
                                 df_feats_icd_transform,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left')

In [7]:
cutoff_y = np.quantile(Outcome_labels['LOS_postoperative'],0.75)
cutoff_y
Outcome_labels['PLOS'] =  (Outcome_labels['LOS_postoperative']>cutoff_y)*1
del Outcome_labels['LOS_postoperative']

Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [8]:
df_icu_graph_features_train = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_icu_graph_features_val = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_icu_graph_features_test = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

len(df_icu_graph_features_train)+len(df_icu_graph_features_val)+len(df_icu_graph_features_test)-len(df_icu_graph_features)

0

In [9]:
df_feats_clinical_icd_ppg_icug_train = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_train_all,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_train,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_train,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_icd_ppg_icug_val = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_val,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_icd_ppg_icug_test = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_test,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


len(df_feats_clinical_icd_ppg_icug_train)+len(df_feats_clinical_icd_ppg_icug_val)+len(df_feats_clinical_icd_ppg_icug_test)-len(df_feats_clinical_icd)

0

In [101]:
y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_test = Outcome_labels_test

X_train = y_train[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_train,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)
X_val = y_val[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_val,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_test = y_test[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_test,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_train_val = pd.concat([X_train,X_val]).reset_index(drop=True)
y_train_val = pd.concat([y_train,y_val]).reset_index(drop=True)

In [102]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 4503),
 (13665, 4503),
 (122983, 4503),
 (13664, 4503),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [103]:
X_train_val.columns.value_counts()

ENCRYPTED_HOSP_ENCOUNTER    1
icd10_transformed_C14       1
icd10_transformed_C20       1
icd10_transformed_C19       1
icd10_transformed_C18       1
                           ..
CPT 1_35560                 1
CPT 1_35563                 1
CPT 1_35565                 1
CPT 1_35566                 1
Node2Vec_127                1
Length: 4503, dtype: int64

# Tuning

In [104]:
del sys.modules["utilities"]
from utilities import *

In [105]:
np.random.seed(666)

In [106]:
a = np.zeros(len(X_train))-1
b = np.ones(len(X_val)) 
pds = PredefinedSplit(np.concatenate([a,b]))

In [107]:
for i , j in pds.split():
    print(len(i))
    print(len(j))

109318
13665


In [108]:
pds.get_n_splits()

1

In [109]:
pipe = Pipeline(steps=[
                       ("imputer", SimpleImputer(missing_values=np.nan, strategy='mean')),
                       ("scaler", MinMaxScaler()),
                       ("classifier", LogisticRegression(max_iter=500,solver='newton-cg')) 
                        ])

In [110]:
clf = BayesSearchCV(estimator = pipe,
                   search_spaces={
                   'classifier__C':Real(1e-6,50,'log-uniform'),
                    'classifier__class_weight':Categorical(['balanced',None]),
                   },
                   n_iter = 30,
                   cv=pds,
                   scoring = 'roc_auc',
                   n_jobs=32,
                   refit=True,
                   verbose=4,
                   random_state=666
                   ) 

In [111]:
clf.fit(X_train_val.iloc[:,1:],y_train_val.iloc[:,1:].values.ravel())

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV 1/1] END classifier__C=0.03689522119794739, classifier__class_weight=balanced;, score=0.855 total time= 5.4min
[CV 1/1] END classifier__C=5.987777606204113, classifier__class_weight=balanced;, score=0.855 total time=19.7min
[CV 1/1] END classifier__C=1.7617470266844044e-06, classifier__class_weight=balanced;, score=0.760 total time= 1.0min
[CV 1/1] END classifier__C=1.1141189663164778e-06, classifier__class_weight=balanced;, score=0.760 total time= 1.7min
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitti

BayesSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
              estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                        ('scaler', MinMaxScaler()),
                                        ('classifier',
                                         LogisticRegression(max_iter=500,
                                                            solver='newton-cg'))]),
              n_iter=30, n_jobs=32, random_state=666, scoring='roc_auc',
              search_spaces={'classifier__C': Real(low=1e-06, high=50, prior='log-uniform', transform='normalize'),
                             'classifier__class_weight': Categorical(categories=('balanced', None), prior=None)},
              verbose=4)

In [112]:
print(clf.best_params_)
best_m = clf.best_estimator_
pred = best_m.predict(X_test.iloc[:,1:])
pred_prob = best_m.predict_proba(X_test.iloc[:,1:])[:,1]
print(min(pred_prob),max(pred_prob))

print('Best Validation AUROC: {}'.format(clf.best_score_))

print('Test AUROC: {}'.format(roc_auc_score(y_test.iloc[:,1:], pred_prob)))
print('Test F1-score: {}'.format(f1_score(y_test.iloc[:,1:], pred)))
print('Test Recall: {}'.format(recall_score(y_test.iloc[:,1:], pred)))
print('Test Precision: {}'.format(precision_score(y_test.iloc[:,1:], pred)))
print('Test Accuracy: {}'.format(accuracy_score(y_test.iloc[:,1:], pred)))

OrderedDict([('classifier__C', 0.31721551510811385), ('classifier__class_weight', 'balanced')])
4.738587923988148e-06 0.9999981249137634
Best Validation AUROC: 0.8617811815582059
Test AUROC: 0.8437799981635907
Test F1-score: 0.6547831253713607
Test Recall: 0.6985294117647058
Test Precision: 0.6161932453589801
Test Accuracy: 0.7873975409836066
[CV 1/1] END classifier__C=0.005510550685485636, classifier__class_weight=balanced;, score=0.835 total time= 4.1min
[CV 1/1] END classifier__C=0.26002267492942843, classifier__class_weight=balanced;, score=0.862 total time=10.3min
[CV 1/1] END classifier__C=0.31721551510811385, classifier__class_weight=balanced;, score=0.862 total time=11.0min
[CV 1/1] END classifier__C=0.051169866640800435, classifier__class_weight=None;, score=0.856 total time= 7.5min


In [13]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [26]:
#experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[2023])